# COMED

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import pymssql

from icap.database.icapdatabase import ICapDatabase

In [2]:
fp = 'icap/database/icapdatabase.json'
conn = ICapDatabase(fp).connect()

In [3]:
# Initialize excel writer
# Use the writer df.to_excel(writer, 'sheet_name')
writer = pd.ExcelWriter('comed.xlsx')

In [4]:
from icap.comed.comed import COMEDInterval

In [5]:
cint = COMEDInterval(conn, premise='9590268008')

## Constants from Test Case

In [6]:
# Weather Normalized Peak Load
# Taken from the test case
COMED_NPL = 20900000;
COMED_CUST_DELTA = 1121264;

## Comed CP Avg Peak Load

In [7]:
'''
def get_comed_cp_avg_peak_load(conn: pymssql.Connection) -> pd.DataFrame:
    query = """
        select
            Cast(CPYearID - 1 as int) as Year,
            AVG(ZonalLoad) * 1000 as AvgCPZonalLoad
        from [COMED_CoincidentPeak]
        group by
            Cast(CPYearID - 1 as int)
    """
    return pd.read_sql(query, conn).set_index('Year')

get_comed_cp_avg_peak_load(conn).to_excel(writer, 'comed_cp_avg_peakload')'''

'\ndef get_comed_cp_avg_peak_load(conn: pymssql.Connection) -> pd.DataFrame:\n    query = """\n        select\n            Cast(CPYearID - 1 as int) as Year,\n            AVG(ZonalLoad) * 1000 as AvgCPZonalLoad\n        from [COMED_CoincidentPeak]\n        group by\n            Cast(CPYearID - 1 as int)\n    """\n    return pd.read_sql(query, conn).set_index(\'Year\')\n\nget_comed_cp_avg_peak_load(conn).to_excel(writer, \'comed_cp_avg_peakload\')'

In [8]:
def get_comed_cp_avg_peak_load(conn: pymssql.Connection)-> pd.DataFrame:
    query = """
        select 
            Cast(CPYearId-1 as int) Year,
            Avg(PJMZonalLoad) * 1000 as AvgCPZonalLoad
        from [CoincidentPeak]
        where UtilityId = 'COMED'
        group by
            Cast(CpYearId-1 as int)
    """
    return pd.read_sql(query, conn).set_index('Year')

In [9]:
comed_cp_avg_peak_load = get_comed_cp_avg_peak_load(conn)

In [10]:
comed_cp_avg_peak_load['Step4Diff'] = COMED_NPL - comed_cp_avg_peak_load['AvgCPZonalLoad'] 

## PJM CP Usage

In [11]:
def get_pjm_cp_usage(conn: pymssql.Connection, premise: str = None) -> pd.DataFrame :
    """Select all records from COMED Hourly
    for PJM coincident peak usage. Filter those values that
    do not possess 5 values per year
    """
    
    # query
    query = """
        select distinct
            h.PremiseId,
            iif(RTrim(p.RateClass)  is null, 'MILES', RTrim(p.RateClass)) as RateClass,
            RTrim(p.DSC) as DSC,
            CAST(cp.CPYearId -1 as INT) as Year, 
            cp.CPDate as CPDatePJM,
            cp.HourEnding as CPHourEnding,
            --cp.HourEnding-1 as ADJCPHourEndingPJM, 
            h.Usage as UsagePJM
        from [COMED_Premise] p
        inner join [HourlyUsage] h on
            p.PremiseId = h.PremiseId
        inner join [CoincidentPeak] cp on
            cp.UtilityId = h.UtilityId and
            cp.CPDate = h.UsageDate and
            cp.HourEnding = h.HourEnding 
        where
            h.UtilityId = 'COMED'
            {prem}
        order by
            h.PremiseId, RateClass, DSC, cp.CPDate
        """
    if premise is not None:
        pjm_cp_query = query.format(prem="and h.PremiseId = '%s'" % premise)
    else:
        pjm_cp_query = query.format(prem="", year="")
    
    # read query
    df = pd.read_sql(pjm_cp_query, conn)
    
    # group by premise
    # create filter for len(usage) != 5
    grp = df.groupby(['PremiseId', 'Year'])['UsagePJM'].agg(
        {'CountPJM': len, 'MeanPJM': np.mean})
    grp.reset_index(inplace=True)
    
    # set `Mean` = np.NaN if `Count` != 5
    missing_data_index = grp[grp['CountPJM'] != 5.0].index
    grp = grp.set_value(missing_data_index, 'MeanPJM', np.nan)
    
    
    
    return pd.merge(df, grp, how='left',
                   on=['PremiseId', 'Year'])

In [12]:
get_pjm_cp_usage(conn).to_excel(writer, 'pjm_cp_usage')

In [13]:
df = get_pjm_cp_usage(conn)

In [14]:
df[df.PremiseId == '1199042022']

,PremiseId,RateClass,DSC,Year,CPDatePJM,CPHourEnding,UsagePJM,MeanPJM,CountPJM
1661,1199042022,MILES,C31,2016,2016-07-25,15.0,740.60,613.692,5.0
1662,1199042022,MILES,C31,2016,2016-07-27,16.0,476.56,613.692,5.0
1663,1199042022,MILES,C31,2016,2016-08-10,16.0,619.22,613.692,5.0
1664,1199042022,MILES,C31,2016,2016-08-11,15.0,589.52,613.692,5.0
1665,1199042022,MILES,C31,2016,2016-08-12,15.0,642.56,613.692,5.0


## COMED CP Usage

In [15]:
def get_comed_cp_usage(conn: pymssql.Connection, premise: str = None) -> pd.DataFrame :
    """Select all records from COMED Hourly
    for COMED coincident peak usage. Filter those values that
    do not possess 5 values per year
    """
    
    # query
    query = """
        select distinct
            h.PremiseId,
            iif(RTrim(p.RateClass)  is null, 'MILES', RTrim(p.RateClass)) as RateClass,
            RTrim(p.DSC) as DSC,
            CAST(cp.CPYearId -1 as INT) as Year, 
            cp.CPDate as CPDateCOMED, 
            cp.HourEnding as CPHourEndingCOMED, 
            h.Usage as UsageCOMED,
            ZonalLoad
        from [HourlyUsage] h
        inner join [COMED_CoincidentPeak] cp on
            cp.UtilityId = h.UtilityId and
            cp.CPDate = h.UsageDate and
            cp.HourEnding = h.HourEnding
        inner join [COMED_Premise] p on
            p.PremiseId = h.PremiseId
        where
            h.UtilityId = 'COMED'
            {prem}
        order by
            h.PremiseId, RateClass, DSC, cp.CPDate
        """
    # format query for single premise
    if premise is not None:
        pjm_cp_query = query.format(prem="and h.PremiseId = '%s'" % premise)
    else:
        pjm_cp_query = query.format(prem="")
    
    # read query
    df = pd.read_sql(pjm_cp_query, conn)
    
    # group by premise
    # create filter for len(usage) != 5
    #df.replace(to_replace=None, value="Miles", inplace=True)
    
    grp = df.groupby(['PremiseId', 'Year', 'RateClass'])['UsageCOMED'].agg(
        {'CountCOMED': len, 'MeanCOMED': np.mean})
    grp.reset_index(inplace=True)
        
    # set `Mean` = np.NaN if `Count` != 5
    missing_data_index = grp[grp['CountCOMED'] != 5.0].index
    grp = grp.set_value(missing_data_index, 'MeanCOMED', np.nan)
    
    return pd.merge(df, grp, how='left',
                   on=['PremiseId', 'Year', 'RateClass'])

In [16]:
get_comed_cp_usage(conn).to_excel(writer, 'comed_cp_usage')

In [17]:
get_comed_cp_usage(conn, premise='1199042022')

,PremiseId,RateClass,DSC,Year,CPDateCOMED,CPHourEndingCOMED,UsageCOMED,ZonalLoad,CountCOMED,MeanCOMED
0,1199042022,MILES,C31,2016,2016-07-11,17.0,587.62,20268.179688,5.0,570.464
1,1199042022,MILES,C31,2016,2016-07-21,18.0,517.93,20296.000000,5.0,570.464
2,1199042022,MILES,C31,2016,2016-08-04,17.0,557.17,20500.960938,5.0,570.464
3,1199042022,MILES,C31,2016,2016-08-11,15.0,589.52,21174.580078,5.0,570.464
4,1199042022,MILES,C31,2016,2016-09-06,17.0,600.08,20949.191406,5.0,570.464


## Load Drop Estimates

In [18]:
def get_comed_load_drop_estimates(conn: pymssql.Connection) -> pd.DataFrame:
    query = """
        select
            Cast(CPYearID -1 as INT) as Year,
            ParameterId,
            (1.0 + ParameterValue/100.0) as LoadDrop
        from [SystemLoad]
        where
            UtilityId = 'COMED' --and
            --ParameterId = 'UFT'
    """
    
    # return pd.read_sql(query, conn).set_index('Year')
    df = pd.read_sql(query, conn)
    return pd.pivot_table(df, index='Year', columns='ParameterId', values='LoadDrop')
get_comed_load_drop_estimates(conn).to_excel(writer, 'load_drop_estimates')

## Utility Factors

In [19]:
def get_comed_utility_factors(conn: pymssql.Connection) -> pd.DataFrame:
    query = """
        select
            Year(StartDate) as Year,
            RTrim(RateClass) as DSC,
            ParameterId, ParameterValue
        from [UtilityParameterValue]
        where
            UtilityId = 'COMED'
            
    """ 
    
    df = pd.read_sql(query, conn)
    
    piv = pd.pivot_table(df, index=['Year', 'DSC'], columns='ParameterId', values='ParameterValue')
    
    return piv.reset_index(level=1)
    
get_comed_utility_factors(conn).to_excel(writer, 'comed_util_factors')

In [20]:
df = get_comed_load_drop_estimates(conn)
#df.reset_index(inplace=True)
#pd.pivot_table(df, columns='ParameterId', values='LoadDrop', index='Year')
df

ParameterId,UFC,UFT
Year,,
2015,1.022679,1.035657
2016,1.022679,1.035657


In [21]:
def compute_acustcpl(conn:pymssql.Connection, premise: str=None)->pd.DataFrame:
    mean_pjm = get_pjm_cp_usage(conn, premise=premise)[['PremiseId', 'Year', 'DSC', 'MeanPJM']] \
        .drop_duplicates() \
        .reset_index()
        
    # System and util factors index on Year
    util = get_comed_utility_factors(conn)
    sys = pd.DataFrame(get_comed_load_drop_estimates(conn)['UFC'])
    
    # Join `util` and `sys` on Year index
    df = pd.merge(util, sys, left_index=True, right_index=True).reset_index()    
   
    # Join mean usage values with utility/system factors
    df = pd.merge(mean_pjm, df, on=['Year', 'DSC']).drop('index', axis=1)
 
    # Compute the AcustPL value
    df['AcustCPL'] = df['MeanPJM'] * df['DistLossFactor'] * df['TransLossFactor'] * df['UFC'] #df['LoadDrop']
    df.set_index(['PremiseId', 'Year'], inplace=True)
    return df

In [22]:
#pd.DataFrame(get_comed_load_drop_estimates(conn)['UFC'])
compute_acustcpl(conn, premise='1199042022')

,,DSC,MeanPJM,DistLossFactor,TransLossFactor,UFC,AcustCPL
PremiseId,Year,,,,,,
1199042022,2016,C31,613.692,1.0629,1.016,1.022679,677.759958


## Compute AcustPL

In [23]:
def compute_acustpl(conn: pymssql.Connection, premise: str=None)->pd.DataFrame:
    # ComedCP unique mean usage values per year; includes np.NaN
    mean_comed = get_comed_cp_usage(conn, premise=premise)[['PremiseId','Year','DSC','MeanCOMED']] \
        .drop_duplicates() \
        .reset_index()  

    # System and util factors index on Year
    util = get_comed_utility_factors(conn)
    sys = pd.DataFrame(get_comed_load_drop_estimates(conn)['UFT'])
    
    # Join `util` and `sys` on Year index
    df = pd.merge(util, sys, left_index=True, right_index=True).reset_index()    
   
    # Join mean usage values with utility/system factors
    df = pd.merge(mean_comed, df, on=['Year', 'DSC']).drop('index', axis=1)
 
    # Compute the AcustPL value
    df['AcustPL'] = df['MeanCOMED'] * df['DistLossFactor'] * df['TransLossFactor'] * df['UFT']#df['LoadDrop']
    df.set_index(['PremiseId', 'Year'], inplace=True)
    return df

In [24]:
compute_acustpl(conn).to_excel(writer, 'ACustPL')

In [25]:
compute_acustpl(conn, premise='1199042022')

,,DSC,MeanCOMED,DistLossFactor,TransLossFactor,UFT,AcustPL
PremiseId,Year,,,,,,
1199042022,2016,C31,570.464,1.0629,1.016,1.035657,638.014127


## AcustCPL

In [26]:
# Import all usage data for utility or premise
# returns pd.DataFrame indexed on [PremiseId, Year]
pjm_cp_usage = get_pjm_cp_usage(conn) \
    .set_index(['PremiseId', 'Year'])

    
# AcustCPL
# AcustCPL = pjm_cp_usage.MeanPJM * (1.0 + SystemLoad.UFT) * UPV.[DistLossFactor, TransLossFactor]
#AcustCPL = pjm_cp_usage['MeanPJM'].drop_duplicates()
AcustCPL = compute_acustcpl(conn)['AcustCPL']

compute_acustcpl(conn).to_excel(writer, 'ACustCPL')

In [27]:
comed_cp_usage = get_comed_cp_usage(conn) \
    .set_index(['PremiseId', 'Year'])
    
AcustPL = compute_acustpl(conn)['AcustPL']

In [28]:
peak_loads = pd.merge(pd.DataFrame(AcustCPL), pd.DataFrame(AcustPL),
                        left_index=True, right_index=True) #\
                #.rename(columns={'MeanPJM': 'AcustCPL'})


#peak_loads.to_excel(writer, 'peak_loads')

In [29]:
    
df = pd.merge(peak_loads, comed_cp_avg_peak_load, left_index=True, right_index=True)

In [30]:
def step7(r: pd.Series) -> np.float32:
    if r.AcustCPL >= r.AcustPL:
        return r.AcustCPL
    
    return (((r.AcustPL - r.AcustCPL) / COMED_CUST_DELTA )* r.Step4Diff) #+ r.AcustCPL
    

df['Step6'] = (df['AcustPL'] - df['AcustCPL']) / COMED_CUST_DELTA
df['Step7'] = df['Step6'] * df['Step4Diff']
 
#df['Step7'] = df.apply(step7, axis=1)

df['ICap'] = df['Step7'] + df['AcustCPL']
#df['ICap'] = df.apply(step7, axis=1)

df.to_excel(writer, 'icap')

In [31]:
df

AcustCPL     AcustPL  AvgCPZonalLoad  Step4Diff  \
PremiseId  Year                                                      
0023101082 2015         NaN         NaN      18880660.0  2019340.0   
           2016         NaN         NaN      19227060.0  1672940.0   
0095686007 2015         NaN         NaN      18880660.0  2019340.0   
           2016         NaN         NaN      19227060.0  1672940.0   
0107454071 2015         NaN         NaN      18880660.0  2019340.0   
           2016         NaN         NaN      19227060.0  1672940.0   
0108142178 2015  375.240448  382.340047      18880660.0  2019340.0   
           2016  350.008786  377.161719      19227060.0  1672940.0   
0123211012 2015   80.520720   82.449700      18880660.0  2019340.0   
           2016         NaN         NaN      19227060.0  1672940.0   
0132350013 2015   57.811392   60.710433      18880660.0  2019340.0   
0136667019 2015   59.735232   65.290859      18880660.0  2019340.0   
           2016   64.809360   67.458292      19227060.0  1672940.0   
0136682007 2015         NaN         NaN      18880660.0  2019340.0   
           2016         NaN         NaN      19227060.0  1672940.0   
0137046041 2015         NaN         NaN      18880660.0  2019340.0   
           2016         NaN         NaN      19227060.0  1672940.0   
0153130093 2015  243.807646  270.860557      18880660.0  2019340.0   
           2016  246.771024  273.992019      19227060.0  1672940.0   
0198310011 2015   60.809376   67.929120      18880660.0  2019340.0   
           2016   67.697124   72.328927      19227060.0  1672940.0   
0216619064 2016   24.939780   27.701735      19227060.0  1672940.0   
0220032048 2015   57.578928   62.863660      18880660.0  2019340.0   
           2016   61.779312   65.510038      19227060.0  1672940.0   
0222651014 2015   60.911580   67.291878      18880660.0  2019340.0   
           2016   58.735236   66.283251      19227060.0  1672940.0   
0269213078 2015    7.362696    8.539846      18880660.0  2019340.0   
           2016    4.272528         NaN      19227060.0  1672940.0   
0272006078 2015         NaN         NaN      18880660.0  2019340.0   
           2016         NaN         NaN      19227060.0  1672940.0   
...                     ...         ...             ...        ...   
8078044030 2015         NaN         NaN      18880660.0  2019340.0   
           2016         NaN         NaN      19227060.0  1672940.0   
8369487005 2015         NaN         NaN      18880660.0  2019340.0   
           2016         NaN         NaN      19227060.0  1672940.0   
8466148003 2015   39.892403   50.382883      18880660.0  2019340.0   
8608329011 2015   91.791216   89.765799      18880660.0  2019340.0   
8661176001 2016         NaN         NaN      19227060.0  1672940.0   
8689683018 2015  177.907376  169.808399      18880660.0  2019340.0   
           2016  163.831330         NaN      19227060.0  1672940.0   
8738194019 2015  104.926136  104.881538      18880660.0  2019340.0   
           2016  100.154936  105.663724      19227060.0  1672940.0   
8750445042 2015   99.520789  107.103735      18880660.0  2019340.0   
           2016   94.848234  104.558742      19227060.0  1672940.0   
8763122003 2015         NaN         NaN      18880660.0  2019340.0   
8768284017 2015  224.991268  236.360114      18880660.0  2019340.0   
           2016  248.496977  256.135620      19227060.0  1672940.0   
8931366011 2015  302.107549  312.440768      18880660.0  2019340.0   
           2016  332.933124  333.190778      19227060.0  1672940.0   
9171474015 2015         NaN         NaN      18880660.0  2019340.0   
           2016         NaN         NaN      19227060.0  1672940.0   
9520311012 2015         NaN         NaN      18880660.0  2019340.0   
9590268008 2015  101.761441  176.919427      18880660.0  2019340.0   
           2016  164.854017  113.833523      19227060.0  1672940.0   
9693060006 2015         NaN         NaN      18880660.0  2019340.0   
           2016         NaN  

In [32]:
#df.ix['9590268008']

#df.ix['1199042022']

<hr>

## Compare against historical

In [33]:

historical_query = """
    select 
        PremiseId, Cast(CPYearID as INT) Year,
        CapacityTagValue as Historical
    from [CapacityTagHistorical]
    where
        UtilityId = 'COMED'
    """

historical = pd.read_sql(historical_query, conn).set_index(['PremiseId','Year'])

In [34]:
def one_kw(r: pd.Series):
    if np.abs(r['ICap'] - r['Historical']) <= 1.0:
        return 'true'
    return 'false'

def passing(r: pd.Series) -> np.int32:
    if abs(r.Variance) <= 2.0 or r['1KW'] == 'true':
        return 1
    return 0

compare = pd.merge(df, historical,
         left_index=True, right_index=True, how='left')

compare['Variance'] = (compare['Historical'] - compare['ICap']) / compare['Historical'] * 100
compare['1KW'] = compare.apply(one_kw, axis=1)

compare['Passing'] = compare.apply(passing, axis=1)

/home/miles/miniconda3/envs/justenergy/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in absolute
  from ipykernel import kernelapp as app


In [35]:
compare.ix['9590268008']

,AcustCPL,AcustPL,AvgCPZonalLoad,Step4Diff,Step6,Step7,ICap,Historical,Variance,1KW,Passing
Year,,,,,,,,,,,
2015,101.761441,176.919427,18880660.0,2019340.0,0.000067,135.355748,237.117189,103.7372,-128.574888,false,0
2016,164.854017,113.833523,19227060.0,1672940.0,-0.000046,-76.123220,88.730797,237.1173,62.579366,false,0


## Output to Excel for analysis

In [36]:
query = "select * from COMED_CoincidentPeak where CPYearID = 2016. order by CPDate"
pd.read_sql(query, conn)

,CPID,CPYearID,ISOID,CPDate,HourEnding,UtilityID,ZonalLoad
0,9,2016.0,PJM,2015-07-17,16.0,COMED,19525.496094
1,7,2016.0,PJM,2015-07-28,16.0,COMED,19765.029297
2,10,2016.0,PJM,2015-09-01,17.0,COMED,19424.433594
3,8,2016.0,PJM,2015-09-02,15.0,COMED,19715.060547
4,6,2016.0,PJM,2015-09-03,17.0,COMED,20162.302734


In [37]:
#writer = pd.ExcelWriter('comed.xlsx')
compare.reset_index(inplace=True)
compare.to_excel(writer, 'variances')

#comed_cp_avg_peak_load.to_excel(writer,'avg_peak_loads')

#AcustCPL.to_excel(writer, 'AcustCPL')
#AcustPL.to_excel(writer, 'ACustPL')
#df.to_excel(writer, 'all_data')
writer.save()

In [38]:
#compare.xs('1199042022', level=0)

In [39]:
compare.to_csv(path_or_buf='~/Projects/BusinessLab/JustEnergy/comed.csv')

# Comparing to Test Cases

In [40]:
import pandas as pd

In [41]:
df = pd.read_csv('/home/miles/Dropbox/iCAP_Project/Results/Analysis/comed.csv')

In [42]:
tc1 = '9590268008'

In [43]:
df[df['PremiseId'] == 9590268008]

,Unnamed: 0,PremiseId,Year,AcustCPL,AcustPL,AvgCPZonalLoad,Step4Diff,ICap,Historical,Variance,1KW,Passing
303,303,9590268008,2015,103.052814,176.919427,1.971846e+07,1.181536e+06,180.889995,103.7372,-74.373315,False,0
304,304,9590268008,2016,166.946048,113.833523,2.063778e+07,2.622176e+05,166.946048,237.1173,29.593476,False,0
